# Experiments with Pymoveit2 Interface

In [1]:
import os
import sys
import yaml
import rclpy
import numpy as np

from rclpy.callback_groups import ReentrantCallbackGroup
from rclpy.node import Node

from pymoveit2 import MoveIt2, MoveIt2State
%load_ext autoreload
%autoreload 2
# message libraries
# from geometry_msgs.msg import PoseStamped, Pose

In [2]:
# init rlcpy
# RUN ONCE
rclpy.init()

## Define Node ETC

In [3]:
# RUN ONCE
node = Node("fanuc_ex")

In [4]:
# Declare parameters for position and orientation
node_parameters = [
    ('position', [0.5, 0.0, 0.25]),
    ('quat_xyzw', [1.0, 0.0, 0.0, 0.0]),
    ('synchronous', True),
    # ('cancel_after_secs', 0.0),
    # ('planner_id', "RRTConnectkConfigDefault"),
    # ('cartesian', False),
    # ('cartesian_max_step', 0.0025),
    # ('cartesian_fraction_threshold', 0.0),
    # ('cartesian_jump_threshold', 0.0),
    # ('cartesian_avoid_collisions', False),
]
for param, val in node_parameters:
    try:
        node.declare_parameter(param, val)
    except:
        continue

In [5]:
# Create callback group that allows execution of callbacks in parallel without restrictions
callback_group = ReentrantCallbackGroup()

## Create Interface for Robot

In [6]:
NOTEBOOK_PATH = f"{os.path.abspath(os.getcwd())}/" + \
    "FanucPythonExperiments/src/ros2_experiments/pymoveit2_exp"

In [7]:
# RUN ONCE
sys.path.append(NOTEBOOK_PATH)

In [8]:
# import robot
from robots import crx20ia
robot = crx20ia()

In [9]:
robot.joint_names

['J1', 'J2', 'J3', 'J4', 'J5', 'J6']

In [10]:
# Create MoveIt 2 interface
moveit2 = MoveIt2(
    node=node,
    joint_names=robot.joint_names,
    base_link_name=robot.base_link_name,
    end_effector_name=robot.end_effector_name,
    group_name=robot.MOVE_GROUP_ARM,
    callback_group=callback_group,
)


## Rclpy Setup

In [11]:
from threading import Thread
import ipyparallel as ipp

In [65]:
with ipp.cluster() as rc:
    # Spin the node in background thread(s) and wait a bit for initialization
    executor = rclpy.executors.MultiThreadedExecutor(2)
    executor.add_node(node)
    executor_thread = Thread(target=executor.spin, daemon=True, args=())
    executor_thread.start()
    node.create_rate(1.0).sleep()

    # Get parameters
    position = node.get_parameter("position").\
        get_parameter_value().double_array_value
    quat_xyzw = node.get_parameter("quat_xyzw").\
        get_parameter_value().double_array_value
    synchronous = node.get_parameter("synchronous").\
        get_parameter_value().bool_value

KeyboardInterrupt: 